# TF-IDF를 활용한 Passage Retrieval 실습

### Requirements

In [2]:
# !pip install datasets
# !pip install transformers

## 데이터셋 준비

KorQuAD train 데이터셋을 search corpus로 활용

In [3]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [4]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비

가장 기본적인 띄워쓰기를 기준으로 token을 나누는 tokenizer를 활용 (성능 향상을 위해 더 세밀한 토크나이저 활용 가능)

In [5]:
tokenizer_func = lambda x: x.split(' ')

In [6]:
tokenizer_func(corpus[0])[:10]

['머서의', '지휘', '첫시즌인', '1965', '-', '66시즌,', '맨체스터', '시티는', '2부리그', '우승을']

## TF-IDF embedding 만들기

Scikit-learn의 TfidfVectorizer를 활용하여 TF-IDF embedding 만들어보기 
(unigram, bigram 활용)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

In [8]:
vectorizer.fit(corpus)
sp_matrix = vectorizer.transform(corpus)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [9]:
sp_matrix.shape

(9606, 1272768)

In [10]:
import pandas as pd
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(10))

             TF-IDF
맨체스터       0.162481
우승을        0.144752
-          0.091626
된다.        0.088557
번째 1부리그    0.083039
4패에        0.083039
4패에 그치는    0.083039
영입에 성공하게   0.083039
첫시즌인       0.083039
첫시즌인 1965  0.083039


## TF-IDF embedding을 활용하여 passage retrieval 실습해보기

앞서 만든 sparse embedding을 활용하여, 실제 passage retrieval 수행해보기

Search query로 KorQuAD train 데이터셋의 질문을 활용하여, 실제 context가 잘 나오는지 확인해보기

In [11]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

Query를 tf-idf vector로 변환

In [12]:
query_vec = vectorizer.transform([query])

In [13]:
query_vec.shape

(1, 1272768)

변환된 query vector를 document들의 vector과 dot product를 수행 => Document들의 similarity ranking을 구함

In [14]:
result = query_vec * sp_matrix.T
result.shape

(1, 9606)

In [15]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

Top-3개의 passage를 retrieve 하고, 실제 ground_truth와 비교해보기

In [16]:
k = 3
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([ 513, 2945, 3179], dtype=int32))

In [17]:
print("[Search query]\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id], "\n")


[Search query]
 호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가? 

[Ground truth passage]
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-2 

## Wikipedia documents에 대해 TF-IDF 실습하기

In [18]:
# First load wikipedia dump
import json

# TODO: Write your own path and load json file
dump_path = "/opt/ml/input/data/wikipedia_documents.json"
with open(dump_path, 'r') as f:
    wiki = json.load(f)

In [19]:
len(wiki), type(wiki)

(60613, dict)

In [44]:
wiki[str(60612)]

{'text': '이 협약은 부당노동행위 제도를 규율하고 있다. 협약 제1조에서 반노동조합 차별행위로부터의 보호를 규정하고, 특히 노동조합에 가입하지 않거나 노동조합에서 탈퇴할 것을 조건으로 고용하는 황견계약과 노동조합원이라거나 노동조합 활동을 했다는 이유로 불이익조치를 취하는 것에 대한 보호를 규정하고 있다.\n\n제2조에서 노동자단체와 사용자단체 사이의 상호간 간섭으로부터 충분한 보호를 하도록 규정한다. 사용자 또는 사용자단체의 지배 하에 둘 목적으로 노동자단체의 설립을 지원하거나 노동자단체에 재정적으로 또는 그 밖의 방법으로 지원하는 것은 간섭행위로 보게 된다.\n\n노동조건을 단체협약으로 규율하도록, 사용자 및 사용자단체와 노동자단체 사이의 자발적 교섭을 위한 기구를 발전시키고 이용을 촉진하는 규정을 담았다.',
 'corpus_source': '위키피디아',
 'url': None,
 'domain': None,
 'title': '단결권 및 단체교섭권 협약',
 'author': None,
 'html': None,
 'document_id': 60612}

In [42]:
# TODO: Generate corpus and Train wiki_vectorizer on wikipedia_documents.json

# wiki_matrix = transformed matrix of documents
wiki_corpus = list()
for i in range(len(wiki)-1):
    wiki_corpus.append(wiki[str(i)]['text'])
wiki_corpus = list(wiki_corpus)
print(len(wiki_corpus))

60613


In [45]:
vectorizer.fit(wiki_corpus)
wiki_matrix = vectorizer.transform(wiki_corpus)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [46]:
# Answer: (60613, 8719114) (num_doc, num_entity)
wiki_matrix.shape

(60613, 8719114)

In [47]:
# TODO: transform query to sparse vector
query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'
query_vec = vectorizer.transform([query])

In [48]:
result = query_vec * wiki_matrix.T
result.shape

(1, 60613)

In [49]:
k = 5
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167, 0.03065025, 0.02977333]),
 array([ 513, 2945, 3179, 3064, 8479], dtype=int32))

In [50]:
print("[Search query]\n", query, "\n")

for i in range(k):
    print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
    doc_id = doc_ids[i]
    print(corpus[doc_id], "\n")

[Search query]
 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은? 

Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-2 passage with score 0.0363
두 사람은 낙담하고, 밴 하우튼의 집을 떠난다. 리더비히는 대신 사과하며 두 사람과 같이 여행을 한다. 세 사람은 안네 프랑크의 집을 방문한다. 집에 계단이 많기 때문에 헤이즐은 힘들게 올라간다. 안네 프랑크의 집 꼭대기에서 헤이즐은 사랑을 느끼고 어거스터스와 로맨틱한 키스를 한다. 두 사람은 호텔로 돌아와 처음으로 밤을 같이 보낸다. 다음날, 어거스터스는 헤이즐에게 자신의 암이 재발했다고 말한다. 인디애나폴리스에 돌아와서 어거스터스의 상태가 더욱 악화되어 갔다. 어거스터스는 중환자실로 보내지며 죽음이 가까운 것을 깨달았다. 어거스터스는 자신의 생전 장례식에 눈 먼 친구 아이작과 헤이즐을 불러 두 사람은 사전에 적은 추도사를 낭독한다. 헤이즐은 밴 하우튼의 소설을 인용하며, 어거스터스와 함께하는 짧은 시간은 무엇과도 바꿀 수 없는 것이라고 말한다. 

Top-3 passage with

In [51]:
# 원본 파일이 수정되는 건지 테스트
"수정 테스트 0502_0234"

'수정 테스트 0502_0234'